In [1]:
from collections import Counter
from itertools import product

In [2]:
with open('../data/Human_BRCA2_orthologues2.fa') as file: righe = file.readlines()

In [3]:
import re
geni = []
for riga in righe:
    riga = riga.rstrip('\n')
    if re.match('^>', riga):
        nuovo_gene = {'sequence': ''}
        geni.append(nuovo_gene)
        nuovo_gene['header'] = re.sub('^>', '', riga)
    elif not re.match('^;', riga):
        nuovo_gene['sequence'] = nuovo_gene['sequence'] + riga

In [12]:
lunghezza = 10
sequenze = [gene['sequence'] for gene in geni][0:5]
num_stringhe = len(sequenze)

In [13]:
def moda(lista):
    c = ''
    conta = Counter(lista)
    val = max(conta.values())
    for c in conta.keys():
        if conta[c] == val:  
            return {'carattere': c, 'valore': val}

Una funzione che riceve un insieme di stringhe e calcola il suo consenso, restituendo un dizionario con chiavi ```stringa``` e ```valore```.

In [14]:
def consenso(stringhe):
    colonne = list(zip(*stringhe))
    caratteri = ''.join([moda(col)['carattere'] for col in colonne])
    valore = sum([moda(col)['valore'] for col in colonne])
    return((caratteri, valore))

In [15]:
def sottostringhe(stringhe, inizio):
     return [x[0][x[1]: x[1] + lunghezza] for x in zip(stringhe,inizio)]

In [16]:
def prossimi(prec, indice, massimi):
    for livello in range(indice, -1, -1):
        prec[livello] += 1
        if prec[livello] < massimi[livello]:
            for i in range(livello + 1, num_stringhe):
                prec[i] = 0
            return prec
    return (None, None)   

Calcola l'array dei punti di inizio, data la stringa di consenso

In [10]:
def distanza(s, t):
    return len([p for p in zip(s,t) if p[0] != p[1]])

In [13]:
print(distanza('ACC', 'AGC'))
print(distanza('ACC', 'ACC'))
print(distanza('ACC', 'CCC'))
print(distanza('CCT', 'CTG'))

1
0
1
2


In [15]:
def punto(stringa, consenso):
    l = len(consenso)
    distanze = [distanza(stringa[i : i + l], consenso) for i in range(len(stringa) - l +1)]
    return distanze.index(min(distanze))

In [16]:
print(punto('ACGACGACGACT', 'GGAG'))
print(punto('ACGACTACGTAC', 'GGAG'))
print(punto('ACGGACTACGAC', 'GGAG'))

1
1
2


In [17]:
def punti(stringhe, consenso):
    return [punto(s, consenso) for s in stringhe]

Calcolo il valore massimo teorico di ogni soluzione dove le prime ```quante``` stringhe sono fissate.

In [17]:
def bound(stringhe, quante):
    return consenso(stringhe[:quante])[1] + (num_stringhe - quante - 1) * lunghezza 

In [18]:
def branch_and_bound():
    punti_inizio = [0] * lunghezza
    massimi = [4] * lunghezza
    motivo = ''
    max_valore = 0
    while(punti_inizio[0] != None):
        (soluzione, valore) = consenso(sottostringhe(sequenze, punti_inizio))
        
        if valore > max_valore:
            max_valore = valore
            motivo = soluzione
            print(soluzione, valore, punti_inizio)
        
        if max_valore == num_stringhe * lunghezza:
            return (motivo, max_valore)

        for buone in range(num_stringhe - 1, -1, -1):
            if bound(sequenze, buone) > valore:
                break
        
        punti_inizio = prossimi(punti_inizio, buone, massimi)
    
        
    return (motivo, max_valore)

In [ ]:
%time print(branch_and_bound())

ATGCCTTTAT 29 [0, 0, 0, 0, 0]
ATGCCTTTAT 30 [0, 0, 0, 0, 164]
ATGCCTTTAT 31 [0, 0, 0, 0, 502]
ATGGCTTTTT 32 [0, 0, 0, 3, 0]
ATGGCTATTG 33 [0, 0, 0, 3, 924]
ATGGCTTTTG 34 [0, 0, 0, 8, 924]


In [ ]:
[len(s) for s in sequenze]